In [2]:
import pandas as pd
import numpy as np

#setting up a file dir so I can reuse it
file_dir = '/Users/richbafa/Desktop/novelis_take_home/take_home_docs/'

#reading in the data for EDA
user_trans =  pd.read_csv(file_dir + 'Candidate_tech_evaluation_candidate_copy_data science_fraud.csv')
ip_to_country_map = pd.read_excel(file_dir + 'Candidate_tech_evaluation_candidate_copy_datascience_IpAddress_to_Country.xlsx')

print('Diemnsions of user transaction is: {0} \nDimensions of IP address to country mapping is: {1}'.format(user_trans.shape, ip_to_country_map.shape))


Diemnsions of user transaction is: (120000, 11) 
Dimensions of IP address to country mapping is: (138846, 3)


## Initial exploration to understand data includes the following question
1. Are there duplicate rows in the user trans data       
2. Is the data truly unique at customer ID level
3. Is device ID unique or it can be shared
4. Is IP address unique
5. Does the join to map each ip address to its corresponding country results in duplicate rows being created or attribution to multiple countries

### Are there duplicate rows

In [3]:
#As seen in the output below, the data does not have duplicates
print('Dimensions of user transactions before dropping duplicates is: {0} \nDimension of user transactions after dropping duplicates is: {1}'.format(user_trans.shape, user_trans.drop_duplicates().shape))

Dimensions of user transactions before dropping duplicates is: (120000, 11) 
Dimension of user transactions after dropping duplicates is: (120000, 11)


### Is the data truly unique at customer ID level

In [4]:
#As seen in the output below, the number of unique user_id is equal to the number of rows in the user trans table
print('number of unique user_id is {}'.format(user_trans[['user_id']].drop_duplicates().size))

number of unique user_id is 120000


### Is device ID unique or it can be shared

In [5]:
#As seen in the output below, there are some devices that were share by different user
#I will explore generating a feature from this and see if that feature can scale
print('number of unique devices is {}'.format(user_trans[['device_id']].drop_duplicates().size))

number of unique devices is 110599


### Is IP address unique

In [17]:
#IP address is not unique as well
print('number of unique devices is {}'.format(user_trans[['ip_address']].drop_duplicates().size))

number of unique devices is 114134


### Does the join to map each ip address to its corresponding country results in duplicate rows being created or attribution to multiple countries
* My compute resource could now handle the cross join between the user trans table and the ip address to country mapping  table. 
* A work around is create a custom ip address to country look up table and left join bac to the trans table with IP address as the key

In [27]:
# Get a list of unique countries to loop over it
countries = list(ip_to_country_map.country.unique())

# DataFrame for unique IP address
ip_table = user_trans[['ip_address']].drop_duplicates()

#for compute efficiency we have to initialize a dict and append each cross join to the dict before converting back to dataframe
results = []
i = 1
total_iter = len(countries)
for country in countries[:4]:
    
    print('iteration {0} of {1}'.format(i, total_iter))
    
    #subset the country map df to include only one country and do the cross join
    country_df = ip_to_country_map[ip_to_country_map['country'] == country]
    
    tmp_cross_join = ip_table.merge(country_df, how = 'cross')
    tmp_cross_join =  tmp_cross_join[(tmp_cross_join.ip_address >= tmp_cross_join.lower_bound_ip_address) & (tmp_cross_join.ip_address <= tmp_cross_join.upper_bound_ip_address)]
    
    # Store the cross join as a list of dict and append it to the master list initialized above
    # Before converting to list of dict, I want to make sure I am dropping duplicates
    tmp = tmp_cross_join[['ip_address', 'country']].drop_duplicates().to_dict('records')
    results.append(tmp)
    
    i += 1

# convert the list of dict bact to Dataframe
ip_look_up = pd.DataFrame.from_dict(results)
ip_look_up.to_csv(file_dir + 'ip_look_up.csv')

# #to map each ip address to the corresponding country, I will need to create a cross join between the trans table and ip to country map table
# user_trans_with_country_map = user_trans.merge(ip_to_country_map, how = 'cross')
# list(user_trans_with_country_map)

iteration 1 of 235
iteration 2 of 235
iteration 3 of 235
iteration 4 of 235


In [6]:
print('number of unique countries is {}'.format(ip_to_country_map[['country']].drop_duplicates().size))

number of unique countries is 235


In [15]:
countries[:3]

['Australia', 'China', 'Japan']

In [18]:
country_df = ip_to_country_map[ip_to_country_map['country'] == 'China']
ip_table = user_trans[['ip_address']].drop_duplicates()
    
tmp_cross_join = ip_table.merge(country_df, how = 'cross')
tmp_cross_join =  tmp_cross_join[(tmp_cross_join.ip_address >= tmp_cross_join.lower_bound_ip_address) & (tmp_cross_join.ip_address <= tmp_cross_join.upper_bound_ip_address)]

tmp_cross_join.head()

,ip_address,lower_bound_ip_address,upper_bound_ip_address,country
49687,1.997833e+09,1997799424,1998061567,China
60515,1.849441e+09,1849163776,1849688063,China
134304,7.166723e+08,716177408,716701695,China
150080,1.931242e+09,1931214848,1931345919,China
182666,6.594343e+08,658505728,660602879,China
